In [1]:
import torch

In [2]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
# Dataset
training_data = datasets.FashionMNIST(
    root='/home/sasatake/repos/python/transformer/data/raw/',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='/home/sasatake/repos/python/transformer/data/raw/',
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
# Dataloader
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader  = DataLoader(test_data, batch_size=64)

In [6]:
# Network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [7]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 64
epoch = 5

In [8]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

In [11]:
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
# train loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # prediction and loss calculation
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:5d} / {size:>5d}]')

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss ,correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
    test_loss /= size
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f} \n')

In [14]:
epoch = 10*2

for t in range(epoch):
    print(f'Epoch {t+1}\n-------------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 1.212539 [    0 / 60000]
loss: 1.298857 [ 6400 / 60000]
loss: 1.064005 [12800 / 60000]
loss: 1.271247 [19200 / 60000]
loss: 1.278541 [25600 / 60000]
loss: 1.140963 [32000 / 60000]
loss: 1.299095 [38400 / 60000]
loss: 1.238576 [44800 / 60000]
loss: 1.262983 [51200 / 60000]
loss: 1.163875 [57600 / 60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 0.019350 

Epoch 2
-------------------------------
loss: 1.171776 [    0 / 60000]
loss: 1.271351 [ 6400 / 60000]
loss: 1.026985 [12800 / 60000]
loss: 1.244752 [19200 / 60000]
loss: 1.253143 [25600 / 60000]
loss: 1.109021 [32000 / 60000]
loss: 1.274185 [38400 / 60000]
loss: 1.214699 [44800 / 60000]
loss: 1.228742 [51200 / 60000]
loss: 1.142289 [57600 / 60000]
Test Error: 
 Accuracy: 56.8%, Avg loss: 0.018937 

Epoch 3
-------------------------------
loss: 1.134681 [    0 / 60000]
loss: 1.248413 [ 6400 / 60000]
loss: 0.993660 [12800 / 60000]
loss: 1.222145 [19200 / 60000]
loss: 1.231952 [25600 / 60000]
l